In [4]:
import tkinter as tk
from tkinter import filedialog as filedialogs
from tkinter.filedialog import askdirectory
from tkinter import *
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import csv
import shutil
import os
option = webdriver.ChromeOptions()
option.add_argument('--headless')
option.add_argument('--disable-gpu')
#driver = webdriver.Chrome('chromedriver')

def selectPath():
    path_ = askdirectory()
    path_var.set(path_)

def id_start():
    window.driver = webdriver.Chrome('chromedriver',chrome_options=option)
    time.sleep(1)
    login_Process();
    time.sleep(1)
    select_Id(id_Entry.get());
    tmpFile = path_Entry.get()+'/{}_data.csv'.format(id_Entry.get())
    try:
        os.remove(tmpFile)
    except OSError as e:
        consoleLog(e)
        txt_msglist.see("end")
    #else:
    #    print("{}_data is deleted successfully")
    prints = '{} 完成 , 若需檢查其他項目請重新輸入編號並開始,結束請關閉。\n'.format(id_Entry.get())
    consoleLog(prints)
    txt_msglist.see("end")
    window.update()
    window.driver.close()

def recordsNum(list_url,id_Num):
    driver = window.driver
    num=0
    with open(path_Entry.get()+'/{}_data.csv'.format(id_Num), 'r+', newline='') as csvfile,open(path_Entry.get()+'/{}_dataOut.csv'.format(id_Num), 'w', newline='') as out:
        writer = csv.writer(out)
        csv_read = list(csv.reader(csvfile))
        writer.writerow(csv_read[0])
        for line in csv_read:
            if(line[7]=='none'):
                if(line[1]!=''): 
                    driver.get(list_url[num])
                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    time.sleep(0.2)
                    while(True):
                        if(len(soup.select_one('table[class="table table-hover table-fixedheader"]').select_one('tbody').select('tr'))>1):
                            break
                        time.sleep(0.1)
                        soup = BeautifulSoup(driver.page_source, 'lxml')
                    #while(True):
                    #    if(soup.select('#timer')[0].text!='' and soup.select('#length')[0].text!=''):
                    #        break
                    #    time.sleep(0.1)
                    #    soup = BeautifulSoup(driver.page_source, 'lxml')
                    #a=0
                    p = soup.select_one('table[class="table table-hover table-fixedheader"]').select_one('tbody').select('tr')
                    #for i in p:
                    #    a=a+1
                    line[7]=len(p)-1
                    num=num+1
                    prints = 'now writing number:'+(str)(num)+' which is '+(str)(line[7])+'\n'
                    consoleLog(prints)
                    txt_msglist.see("end")
                    window.update()
                    time.sleep(0.1)
                writer.writerow(line)

def login_Process():
    driver = window.driver
    prints = 'opening chrome..\n'
    consoleLog(prints)
    window.update()
    driver.get('http://address/login') 
    prints = 'typing account & pw\n'
    consoleLog(prints)
    window.update()
    user_name = 'account' #input('帳號:')
    password = 'pwd' #input('密碼:')

    elem = driver.find_element_by_name("name")
    elem.clear()
    elem.send_keys(user_name)

    elem = driver.find_element_by_name("password")
    elem.clear()
    elem.send_keys(password)
    driver.find_element_by_xpath("//button[@type='submit']").click()
    prints = 'successfully logined to account : ' + user_name + '!' + '\n'
    consoleLog(prints)
    window.update()
    
def select_Id(id_Num):
    driver = window.driver
    prints = 'now start to do num : '+ (str)(id_Num) + '\n'
    consoleLog(prints)
    window.update()
    url = 'http://address/tasks/{}/hits/list?page={}'
    driver.get(url.format(id_Num,1))
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    counts = 0
    lists = ['id','href','condition','worder','workTime','editedTime','Reviewer','recordsNum']
    k = soup.select('ul[class="pagination"]')
    for i in k[0].select('a'):
        #print(i.text)
        if(i.text=='»'):
            break
        counts=(int)(i.text)
    prints = '總頁數 : '+(str)(counts) + '\n'    #總頁數
    consoleLog(prints)
    txt_msglist.see("end")
    window.update()
    list_url = []
    with open(path_Entry.get()+'/{}_data.csv'.format(id_Num), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for pages in range(counts):
            driver.get(url.format(id_Num,pages+1))
            soup = BeautifulSoup(driver.page_source, 'lxml')
            count = -1
            prints = '第 '+ (str)(pages+1)+' 頁\n'
            consoleLog(prints)
            window.update()
            lists = ['id','href','condition','worder','workTime','editedTime','Reviewer','recordsNum']
            for i in soup.select('tr'):
                #href = i.select_one('td') #抓ID  
                types = 1   #全類別共八個資訊 {id,href,condition,worder,workTime,editedTime,Reset,Reviewer}
                #print(pages+1)
                for j in i.select('td'):
                    if(types%8==1):  #id
                        lists[0] = j.text
                        #print('id = '+lists[0])
                    elif(types%8==2 and len(j.text)<100):  #href
                        #print(j.text+(str)(len(j.text)))
                        lists[1] = j.select_one('a').get('href')
                        list_url.append(lists[1])
                        #print('網址 = '+lists[1]) #網址
                    elif(types%8==2 and len(j.text)>=100):
                        lists[1] = ''
                        #print(j.text+(str)(len(j.text)))
                    elif(types%8==3):  #Condition
                        lists[2] = j.text
                        #print('Condition = '+lists[2])
                    elif(types%8==4):  #Worker
                        lists[3] = j.text
                        #print('Worker = '+lists[3])
                    elif(types%8==5):  #工作時間
                        k = j.text.split(" ")
                        lists[4] = k[0]
                        #print('工作時間 = '+lists[4] +' mins')
                    elif(types%8==6):  #最後修改時間
                        lists[5] = j.text
                        #print('最後修改時間 = '+lists[5])
                    elif(types%8==0):  #Reviewed By
                        lists[6] = j.text.strip()
                        #print('Reviewed By = '+lists[6])
                    if(cb_one_var.get()==1):
                        lists[7] = '1'
                    if(cb_one_var.get()==0):
                        lists[7] = 'none'
                    types=types+1
                count=count+1
                if(lists[0]=='id'):
                    continue
                writer.writerow(lists)
            prints = '此頁總數 : ' +(str)(count) + '\n'
            consoleLog(prints)
            window.update()
    prints = 'starting counting records ..\n'
    consoleLog(prints)
    window.update()
    if(cb_one_var.get()==0):
        recordsNum(list_url,id_Num)
    if(cb_one_var.get()==1):
        shutil.copyfile(path_Entry.get()+'/{}_data.csv'.format(id_Num), path_Entry.get()+'/{}_dataOut.csv'.format(id_Num)) 
    
def pauseEnd():
    try:
        window.driver.close()
    except OSError as e:
        print(e)
    try:
        sys.exit(0)
    except:
        print('Program is dead.')
    finally:
        print('clean-up')
        prints = '程序已終止,可直接關閉程式或重新執行\n'
        consoleLog(prints)

def consoleLog(prints):
    txt_msglist.config(state=NORMAL)
    txt_msglist.insert(END,prints,'black')
    txt_msglist.config(state=DISABLED)
###############################################


#####################GUI布局##########################


window = tk.Tk()
window.title('任務狀況爬蟲 to CSV')
window.geometry('560x300')
#建構window

id_var = tk.StringVar()
id_Label = tk.Label(window, textvariable=id_var,  font=('Arial', 16), width=18, height=2)
id_Label.place(x=0, y=0, anchor='nw')
id_var.set('請輸入任務id(共三碼)')
id_var = tk.StringVar()
id_Label = tk.Label(window, textvariable=id_var,  font=('Arial', 16), width=18, height=2)
id_Label.place(x=0, y=70, anchor='nw')
id_var.set('檔案輸出目的地')
#文字框 (anchor為茅點)

id_Entry = tk.Entry(window, show=None, font=('Arial', 14), width=7)  # 顯示成明文形式
id_Entry.place(x=0, y=50, anchor='nw')
#id輸入框 

path_var = tk.StringVar()
path_Entry = tk.Entry(window, textvariable=path_var,show=None, font=('Arial', 10),width=35,state = 'disabled')  # 顯示成明文形式
path_Entry.place(x=0, y=110, anchor='nw')
#檔案存檔位置顯示框 


f_msglist = Frame(height = 200,width = 190, bg='white')
f_msglist.place(x=280, y=0, anchor='nw')
txt_msglist = Text(f_msglist,height = 22 , width=38)
txt_msglist.grid()
txt_msglist.config(state=NORMAL)    #插入新文字前,先將狀態切為可編輯
txt_msglist.insert(END,'1.輸入3碼編號\n','black')
txt_msglist.insert(END,'2.選擇檔案儲存目的地\n','black')
txt_msglist.insert(END,'3.按下開始.選擇檔案儲存目的地\n','black')
txt_msglist.config(state=DISABLED) #插入文字後,將狀態切為不可編輯,

btn_One = tk.Button(window, text='開始', font=('Arial', 12), width=10, height=1, command=id_start)
btn_One.place(x=0, y=180, anchor='nw')
btn_Two = tk.Button(window, text='瀏覽', font=('Arial', 12), width=10, height=1, command=selectPath)
btn_Two.place(x=0, y=135, anchor='nw')
btn_Tri = tk.Button(window, text='結束', font=('Arial', 12), width=10, height=1, command=pauseEnd)
btn_Tri.place(x=120, y=180, anchor='nw')
#button 
cb_one_var = tk.IntVar()
cb_one = tk.Checkbutton(window, text='音檔皆僅有一筆音訊檔', variable=cb_one_var, onvalue=1, offvalue=0)
cb_one.place(x=0, y=220, anchor='nw')
#checkbutton

window.mainloop()